<a href="https://colab.research.google.com/github/andrecarlos26/Projeto-Interdisciplinar-SI-2/blob/main/Pr%C3%A9_processamento_e_Tratamento_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import statistics as sts
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder


caminho = '/content/drive/My Drive/microdados_enem_2019/MICRODADOS_ENEM_2019_PE.csv'
dados_pe = pd.read_csv(caminho)

In [ ]:
dados_pe.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001044305,2019,2610004,Palmares,26,PE,21,F,0,0,1,2611606.0,Recife,26.0,PE,1,4,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,...,ACBCEEBCCBABADBBEACABBEECCCAACECADCBBEABBEADE,EBBAD99999BACBCCADEBAAADEABCDECBDBADBBBAAECACC...,AEEADEEBAECBADBDCEDBAAADBDDBBEDCEBEADEECBCBAB,0,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,ACACEEBCCBABADBBBACDBBACCCCADCEBADCBEEDBBEADB,EBBADCABDABACBCEBDEEAAADDBECDECDDBADBCDAAECBCC...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,200.0,200.0,160.0,160.0,160.0,880.0,E,H,F,F,6,E,A,D,E,B,A,B,B,B,A,B,A,A,C,A,A,E,A,C,B
1,190001051159,2019,2604007,Carpina,26,PE,28,M,1,2,1,2608909.0,Limoeiro,26.0,PE,1,6,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,D,C,B,1,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
2,190001058610,2019,2614105,Sertânia,26,PE,26,M,1,3,1,2614105.0,Sertânia,26.0,PE,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,F,E,D,4,G,A,B,C,B,A,B,A,A,A,B,A,A,B,A,A,D,A,B,B
3,190001070444,2019,2610509,Passira,26,PE,33,F,1,1,0,NaN,NaN,NaN,NaN,1,13,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,G,E,D,5,J,D,D,E,C,C,D,B,C,B,C,B,B,E,B,B,E,B,E,B
4,190001087987,2019,2610707,Paulista,26,PE,34,F,1,3,1,1302603.0,Manaus,13.0,AM,1,13,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,D,C,B,3,B,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A


# Tratamento de alunos faltosos e eliminados

In [ ]:
# Encontrar index da feature
dados_pe.columns.get_loc('NU_NOTA_REDACAO')

110

In [ ]:
# Remoção dos faltosos e eliminados do Primeiro dia (Redação, Linguagens e Humanas)
dados_pe.dropna(subset=['NU_NOTA_REDACAO'], inplace=True)

In [ ]:
# Remoção dos faltosos e eliminados do Segundo Dia (Matemática e Natureza)
dados_pe.dropna(subset=['NU_NOTA_MT'], inplace=True)

In [ ]:
# Tratamento de algumas colunas NaN de dados do participante que vão ser utilizadas no algoritmo de predição
moda = int(sts.mode(dados_pe['CO_MUNICIPIO_NASCIMENTO']))
dados_pe['CO_MUNICIPIO_NASCIMENTO'].fillna(moda, inplace=True)

In [ ]:
moda = sts.mode(dados_pe['NO_MUNICIPIO_NASCIMENTO'])
dados_pe['NO_MUNICIPIO_NASCIMENTO'].fillna(moda, inplace=True)

In [ ]:
moda = int(sts.mode(dados_pe['CO_UF_NASCIMENTO']))
dados_pe['CO_UF_NASCIMENTO'].fillna(moda, inplace=True)

In [ ]:
moda = sts.mode(dados_pe['SG_UF_NASCIMENTO'])
dados_pe['SG_UF_NASCIMENTO'].fillna(moda, inplace=True)

In [ ]:
dados_pe.shape

(202753, 136)

Após a remoção dos alunos faltosos de eliminados a quantidade de instâncias do dataset foi reduzida a 202753

# Criação das colunas NU_MEDIA e CATEGORIA_MEDIA

In [ ]:
notas = dados_pe.iloc[:,  np.r_[91:95,110]].values

In [ ]:
media = []

for i in notas:
  media.append(sts.mean(i))

In [ ]:
dados_pe['NU_MEDIA'] = media

In [ ]:
# obtenção dos valores dos quantis
quantil_25 = dados_pe['NU_MEDIA'].quantile(0.25)
quantil_50 = dados_pe['NU_MEDIA'].quantile(0.50)
quantil_75 = dados_pe['NU_MEDIA'].quantile(0.75)
print(quantil_25)
print(quantil_50)
print(quantil_75)

458.5
508.46
568.4200000000001


In [ ]:
tipo_media = []
for i in media:
  if (i <= quantil_25):
    tipo_media.append('D')
  elif (i > quantil_25 and i <= quantil_50):
    tipo_media.append('C')
  elif (i > quantil_50 and i <= quantil_75):
    tipo_media.append('B')
  else:
    tipo_media.append('A')

In [ ]:
dados_pe['TP_MEDIA'] = tipo_media

In [ ]:
pd.set_option('max_columns', 138)
dados_pe.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_TEMPO_ADICIONAL,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOADA,IN_PROVA_DEITADO,IN_MOBILIARIO_OBESO,IN_LAMINA_OVERLAY,IN_PROTETOR_AURICULAR,IN_MEDIDOR_GLICOSE,IN_MAQUINA_BRAILE,IN_SOROBAN,IN_MARCA_PASSO,IN_SONDA,IN_MEDICAMENTOS,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,NU_MEDIA,TP_MEDIA
0,190001044305,2019,2610004,Palmares,26,PE,21,F,0,0,1,2611606.0,Recife,26.0,PE,1,4,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1302603,Manaus,13,AM,1,1,1,1,503.0,509.0,514.0,515.0,619.6,674.7,629.1,705.7,ABECDCDEADCACDDABBECCEDBEAAEBACBBDCEBCDEADDDC,ACBCEEBCCBABADBBEACABBEECCCAACECADCBBEABBEADE,EBBAD99999BACBCCADEBAAADEABCDECBDBADBBBAAECACC...,AEEADEEBAECBADBDCEDBAAADBDDBBEDCEBEADEECBCBAB,0,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,ACACEEBCCBABADBBBACDBBACCCCADCEBADCBEEDBBEADB,EBBADCABDABACBCEBDEEAAADDBECDECDDBADBCDAAECBCC...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,200.0,200.0,160.0,160.0,160.0,880.0,E,H,F,F,6,E,A,D,E,B,A,B,B,B,A,B,A,A,C,A,A,E,A,C,B,701.82,A
5,190001095236,2019,2604304,Cedro,26,PE,23,M,1,3,1,2604304.0,Cedro,26.0,PE,1,5,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2307106,Jardim,23,CE,1,1,1,1,505.0,508.0,512.0,518.0,518.7,450.1,364.6,453.3,AACBBEBDACCBCECCEEBBCCEBDDAEABBEBECCDADDDEDBD,DAEAEAACBCBADCBBDDBCEDECECAAEACCECEECECBECAAD,99999CDCEBECBBCBDDCDACEBACAAEBCCBEAEBCEAADDADD...,BCCBACCEEDABADBCDBCADBBDDBADBEDBCEBDABCDBDCAD,1,DADCCEBBCCACBEEBEEBACBCDDDDADBCBBCEAEADEADAAE,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,DBEBACABCDBABECEEEDCBDCCEDCDABEDAADDDECACAECB,1.0,60.0,120.0,80.0,100.0,40.0,400.0,E,C,A,A,4,B,A,B,C,B,B,B,A,B,A,A,A,A,B,A,A,C,A,B,B,437.34,D
7,190001098520,2019,2615607,Trindade,26,PE,19,M,1,2,1,2615607.0,Trindade,26.0,PE,1,2,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2208007,Picos,22,PI,1,1,1,1,505.0,509.0,514.0,518.0,629.8,609.5,581.9,676.8,AADABEDBDCACBBEABEBDCECDBACBDBBAABECEADEDDABE,ACACEEBCAAACADBBBACABBACCEBABECCBECBCEDBAAADD,99999DABDABBCCCCECEBDAADDDEEDECDDBADACCDCDCEEC...,DDABABCACDBACEBECEECABEDAEBBAEECAADAAEDDBEDEB,1,DADCCEBBCCACBEEBEEBACBCDDDDADBCBBCEAEADEADAAE,ACACEEBCCBABADBBBACDBBACCCCADC

In [ ]:
dados_pe.groupby('TP_MEDIA').size()

TP_MEDIA
A    50688
B    50678
C    50688
D    50699
dtype: int64

# Verificando se existem instâncias fora do domínio

In [ ]:
group_cor_raca = dados_pe.groupby(['TP_COR_RACA']).size()
group_cor_raca

TP_COR_RACA
0      4692
1     59917
2     23396
3    106420
4      5727
5      2601
dtype: int64

In [ ]:
group_no_mun_res = dados_pe.groupby(['NO_MUNICIPIO_RESIDENCIA']).size()
group_no_mun_res.sort_values(ascending=False)

NO_MUNICIPIO_RESIDENCIA
Recife                     42085
Jaboatão dos Guararapes    15187
Petrolina                  11224
Olinda                      9322
Paulista                    8101
                           ...  
Solidão                      104
Itacuruba                    102
Manari                        82
Ingazeira                     39
Fernando de Noronha           29
Length: 185, dtype: int64

In [ ]:
group_no_mun_nas = dados_pe.groupby(['NO_MUNICIPIO_NASCIMENTO']).size()
group_no_mun_nas

NO_MUNICIPIO_NASCIMENTO
Abaetetuba                  5
Abaré                      26
Abreu e Lima             1372
Acajutiba                   1
Acarape                     1
                         ... 
Águas Belas               505
Águas Lindas de Goiás       4
Águas de Lindóia            1
Águas de São Pedro          2
Álvares Machado             1
Length: 1390, dtype: int64

In [ ]:
group_tp_ano_con = dados_pe.groupby(['TP_ANO_CONCLUIU']).size()
group_tp_ano_con

TP_ANO_CONCLUIU
0     93347
1     27233
2     17636
3     12314
4      8958
5      7003
6      5320
7      4233
8      3506
9      3012
10     2607
11     2047
12     1839
13    13698
dtype: int64

In [ ]:
group_q001 = dados_pe.groupby(['Q001']).size()
group_q001

Q001
A    11722
B    46480
C    29615
D    21965
E    55944
F    12614
G     7829
H    16584
dtype: int64

In [ ]:
group_q005 = dados_pe.groupby(['Q005']).size()
group_q005

Q005
1      3114
2     22003
3     50267
4     70136
5     35624
6     12776
7      5177
8      2130
9       746
10      425
11      155
12       97
13       42
14       24
15        9
16       10
17        3
18        6
19        1
20        8
dtype: int64

# Verificando se features que vão ser utilizadas possuem NaN's

In [ ]:
pd.set_option('max_rows',300)
dados_pe.isnull().sum()

NU_INSCRICAO                        0
NU_ANO                              0
CO_MUNICIPIO_RESIDENCIA             0
NO_MUNICIPIO_RESIDENCIA             0
CO_UF_RESIDENCIA                    0
SG_UF_RESIDENCIA                    0
NU_IDADE                            0
TP_SEXO                             0
TP_ESTADO_CIVIL                     0
TP_COR_RACA                         0
TP_NACIONALIDADE                    0
CO_MUNICIPIO_NASCIMENTO             0
NO_MUNICIPIO_NASCIMENTO             0
CO_UF_NASCIMENTO                    0
SG_UF_NASCIMENTO                    0
TP_ST_CONCLUSAO                     0
TP_ANO_CONCLUIU                     0
TP_ESCOLA                           0
TP_ENSINO                       77189
IN_TREINEIRO                        0
CO_ESCOLA                      151981
CO_MUNICIPIO_ESC               151981
NO_MUNICIPIO_ESC               151981
CO_UF_ESC                      151981
SG_UF_ESC                      151981
TP_DEPENDENCIA_ADM_ESC         151981
TP_LOCALIZAC

# Extração de features selecionadas para predição

In [ ]:
dados_pe.columns.get_loc('NO_MUNICIPIO_RESIDENCIA')

3

In [ ]:
dados_pe.columns.get_loc('NU_IDADE')

6

In [ ]:
dados_pe.columns.get_loc('TP_COR_RACA')

9

In [ ]:
dados_pe.columns.get_loc('NO_MUNICIPIO_NASCIMENTO')

12

In [ ]:
dados_pe.columns.get_loc('TP_ANO_CONCLUIU')

16

In [ ]:
dados_pe.columns.get_loc('Q001')

111

In [ ]:
dados_pe.columns.get_loc('Q006')

116

In [ ]:
dados_pe.columns.get_loc('TP_MEDIA')

137

In [ ]:
dados_pe.columns.get_loc('NU_MEDIA')

136

In [ ]:
dados_selecionados = dados_pe.iloc[:,  np.r_[3,6,9,12,16,111:117,136,137]]
dados_selecionados.head(10)

,NO_MUNICIPIO_RESIDENCIA,NU_IDADE,TP_COR_RACA,NO_MUNICIPIO_NASCIMENTO,TP_ANO_CONCLUIU,Q001,Q002,Q003,Q004,Q005,Q006,NU_MEDIA,TP_MEDIA
0,Palmares,21,0,Recife,4,E,H,F,F,6,E,701.82,A
5,Cedro,23,3,Cedro,5,E,C,A,A,4,B,437.34,D
7,Trindade,19,2,Trindade,2,E,C,B,A,6,C,655.60,A
9,Petrolina,25,3,Paulistana,6,H,B,A,A,4,B,526.34,B
12,Ipubi,16,1,Crato,0,C,F,D,D,5,D,651.76,A
13,Cedro,17,1,Cedro,0,B,B,F,A,3,B,634.92,A
16,Bodocó,22,0,Ouricuri,6,A,D,A,A,4,B,539.90,B
17,Serrita,19,3,Serrita,3,B,E,A,B,4,B,551.50,B
19,Ipubi,19,3,Ipubi,2,F,F,D,D,5,E,587.12,A
20,Cedro,17,1,Cedro,0,B,E,A,A,3,B,494.44,C


In [ ]:
dados_selecionados.index = range(len(dados_selecionados.index))
dados_selecionados

,NO_MUNICIPIO_RESIDENCIA,NU_IDADE,TP_COR_RACA,NO_MUNICIPIO_NASCIMENTO,TP_ANO_CONCLUIU,Q001,Q002,Q003,Q004,Q005,Q006,NU_MEDIA,TP_MEDIA
0,Palmares,21,0,Recife,4,E,H,F,F,6,E,701.82,A
1,Cedro,23,3,Cedro,5,E,C,A,A,4,B,437.34,D
2,Trindade,19,2,Trindade,2,E,C,B,A,6,C,655.60,A
3,Petrolina,25,3,Paulistana,6,H,B,A,A,4,B,526.34,B
4,Ipubi,16,1,Crato,0,C,F,D,D,5,D,651.76,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202748,Jupi,28,3,Jupi,10,B,B,A,A,5,A,448.04,D
202749,Olinda,19,3,Recife,3,E,E,B,B,2,C,545.56,B
202750,Afogados da Ingazeira,21,1,Afogados da Ingazeira,5,C,F,A,D,8,E,562.10,B
202751,Recife,44,1,Recife,8,A,A,B,B,3,B,486.16,C


# Exportação do novo dataframe csv

In [ ]:
dados_selecionados.to_csv('/content/drive/My Drive/microdados_enem_2019/DADOS_TRATADOS.csv', index=False) 